In [ ]:
import argparse
import cv2
from SinterParticleUtil import ImageAnalysis


# function to read
def read_image(path):
    image = cv2.imread(path)
    return image


def main(fn1, fn2, fn3, fn4):
    """
    This function describes the steps involved in the analysis of sinter particle.
    It consists of two parts. First we set the path of 4 files to be read,
    using this path image is read and finally it call 'ImageAnalysis' module present in 'SinterParticleUtil'
    """

    # Step1: Read the image in numpy array
    img1 = read_image(fn1)
    img2 = read_image(fn2)
    img3 = read_image(fn3)
    img4 = read_image(fn4)

    # Step2: Call ImageAnalysis module to compute histogram of sinter particle
    print("Start.....")
    bins, data = ImageAnalysis(img1, img2, img3, img4)
    # print("Data=", data)

    print("...  Task Completed  ...")


if __name__ == "__main__":
    # STEP: 0 command line arguments sample command to run on terminal: python3 main.py
    # -fn1="img/Cycle77_bottom_2.jpg" -fn2="img/Cycle77_left_2.jpg" -fn3="img/Cycle77_right_2.jpg"
    # -fn4="img/Cycle77_top_2.jpg"
    parser = argparse.ArgumentParser(description='Sinter particle analysis')
    parser.add_argument('-fn1', default="img/Cycle77_bottom_2.jpg", help='image path relative (1st image)', type=str)
    parser.add_argument('-fn2', default="img/Cycle77_left_2.jpg", help='image path relative (2nd image)', type=str)
    parser.add_argument('-fn3', default="img/Cycle77_right_2.jpg", help='image path relative (3rd image)', type=str)
    parser.add_argument('-fn4', default="img/Cycle77_top_2.jpg", help='image path relative (4th image)', type=str)
    args = parser.parse_args()

    # Step1: Set the path of image to read

    fn1 = args.fn1
    fn2 = args.fn2
    fn3 = args.fn3
    fn4 = args.fn4

    # Disable these four line if parameters are passed from command line
    # fn1 = 'img/Cycle77_bottom_2.jpg'
    # fn2 = 'img/Cycle77_left_2.jpg'
    # fn3 = 'img/Cycle77_right_2.jpg'
    # fn4 = 'img/Cycle77_top_2.jpg'

    print(args.fn1)  # display the first file name

    # Step2: call main function
    main(fn1, fn2, fn3, fn4)


AttributeError: ignored

In [ ]:
# import the necessary packages

import shutil
import os
import yaml
from skimage.filters import threshold_mean
import cv2
from skimage.morphology import remove_small_objects, binary_erosion, area_opening, binary_opening, disk, binary_closing, \
    dilation, erosion
import matplotlib.pyplot as plt
from queue import Queue
from threading import Thread
import pickle
import numpy as np
import torch


def GetFileList(dirName1, fname2FullPathDic):
    cnt = 0
    for path, subdirs, files in os.walk(dirName1):

        for name in files:
            extension = os.path.splitext(name)[1]
            if (extension == '.jpg'):
                if name not in fname2FullPathDic:
                    value = os.path.join(path, name)
                    fname2FullPathDic[cnt] = value


def readVar(fn):
    # Open the file in binary mode
    with open(fn, 'rb') as file:
        # Call load method to deserialze
        myvar = pickle.load(file)

    return myvar


def imreconstruct(marker: np.ndarray, mask: np.ndarray, radius: int = 1):
    """Iteratively expand the markers white keeping them limited by the mask during each iteration.

    :param marker: Grayscale image where initial seed is white on black background.
    :param mask: Grayscale mask where the valid area is white on black background.
    :param radius Can be increased to improve expansion speed while causing decreased isolation from nearby areas.
    :returns A copy of the last expansion.
    Written By Semnodime.
    """
    kernel = np.ones(shape=(radius * 2 + 1,) * 2, dtype=np.uint8)
    i = 0
    while True:
        i = i + 1
        expanded = cv2.dilate(src=marker, kernel=kernel)
        cv2.bitwise_and(src1=expanded, src2=mask, dst=expanded)

        # Termination criterion: Expansion didn't change the image at all
        if (marker == expanded).all() or i > 10:
            return expanded
        marker = expanded


def imcomplement(img):
    table = np.array([255 - i for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)  # Use OpenCV's lookup table function


def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # status_for_WindowSize

    ratio = 1894 / 1045

    if status_for_WindowSize == 'Normal':
        maxHeight = max(int(heightA), int(heightB))
        maxWidth = max(int(widthA), int(widthB))

    elif status_for_WindowSize == 'Reduced':
        maxWidth = 500
        maxHeight = int(maxWidth * ratio)
    else:
        maxWidth = max(int(widthA), int(widthB))
        maxHeight = int(maxWidth * ratio)

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    # print('image, M, (maxWidth, maxHeight)=',image, M, (maxWidth, maxHeight))
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped


def plot_Histogram_intermsof_weight(data, label, fname):
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(8, 5))
    ax = fig.add_axes([0, 0, 1, 1])
    bins = ['<5mm', '5 - 10mm', '10 -15mm', '15 - 25mm', '25 - 40mm', '40 - 50mm', '> 50mm']
    # data=out2[8]
    ax.bar(bins, data)
    plt.ylabel('Weights ( in Kg )', fontsize=12)
    plt.xlabel('Bins used for sinter particle ( in mm )', fontsize=12)
    plt.title(label, fontsize=18)
    # plt.show()
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    return plt, bins, data


def displayImageAndPerspectiveCorrection(dst, image):
    cv2.namedWindow('Original', cv2.WINDOW_NORMAL)
    cv2.imshow("Original", image)
    cv2.namedWindow('Warped', cv2.WINDOW_NORMAL)
    cv2.imshow("Warped", dst)


def combine_image(dst1, dst2, dst3, dst4):
    mm1 = np.minimum(dst1, dst2)
    mm2 = np.minimum(dst3, dst4)
    mm = np.minimum(mm1, mm2)
    return mm


def Append_log(List, flog):
    # Import writer class from csv module
    from csv import writer

    # List
    # List = [6, 'William', 5532, 1, 'UAE']

    # Open our existing CSV file in append mode
    # Create a file object for this file
    with open(flog, 'a', newline='') as f_object:
        # Pass this file object to csv.writer()
        # and get a writer object
        writer_object = writer(f_object)

        # Pass the list as an argument into
        # the writerow()
        writer_object.writerow(List)

        # Close the file object
        f_object.close()


def plot_Unequal_Bin_Histogram(bins, xLabel, yLabel, Data, conv, fname):
    # https://stackoverflow.com/questions/33497559/display-a-histogram-with-very-non-uniform-bin-widths
    # bins = [0,5,10,15,25,40,50,500] # your bins
    # data=np.array(coll_major_axis)*conv
    data = np.array(Data) * conv
    # data=np.array(coll_major_axis)
    # print(data)
    hist, bin_edges = np.histogram(data, bins)  # make the histogram

    fig, ax = plt.subplots()

    # Plot the histogram heights against integers on the x axis
    ax.bar(range(len(hist)), hist, width=1)

    # Set the ticks to the middle of the bars
    ax.set_xticks([0.5 + i for i, j in enumerate(hist)])

    # Set the xticklabels to a string that tells us what the bin edges were
    ax.set_xticklabels(['{} - {}'.format(bins[i], bins[i + 1]) for i, j in enumerate(hist)])
    # plt.ylabel('Frequency');
    # plt.xlabel('Major_axis in terms of mm');
    plt.ylabel(xLabel)
    plt.xlabel(yLabel)
    plt.title('Histogram')
    # plt.save()
    # plt.savefig(fname)

    return hist


def imcomplement(img):
    table = np.array([255 - i for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)  # Use OpenCV's lookup table function


################################################
with open(r'config.yaml') as file:
    conf_list = yaml.load(file, Loader=yaml.FullLoader)


def readImage(fname):
    image = cv2.imread(fname)
    return image


def PerspectiveCorrectionMultiThread(image):
    # fn = mySet + fname
    # image = cv2.imread(fn)

    pts = np.float32([[2150, 0150.0], [4450.3, 150.0], [5750.6, 4000], [0550.5, 4000]])
    # apply the four point tranform to obtain a "birds eye view" of

    if status_for_crop == False:
        warped = four_point_transform(image, pts)
        warped1 = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

        crop = warped1
    else:
        warped1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        crop = warped1[DescCrop_y:DescCrop_y + DescCrop_h, DescCrop_x:DescCrop_x + DescCrop_w]

    return crop, image


def CreateDir(parent_dir, directory):
    # Path
    path = os.path.join(parent_dir, directory)

    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)  # Removes all the subdirectories! mks attention required
        os.makedirs(path)


def data_x_normalize(data):
    # example of a standardization
    from numpy import asarray
    from sklearn.preprocessing import StandardScaler, QuantileTransformer
    # define data
    #     data=x
    # print('x=',data)
    # define standard scaler
    # scaler = StandardScaler()
    scaler = QuantileTransformer()

    # transform data
    scaler.fit_transform(data)
    scaled = scaler.fit_transform(data)
    # print('scaled X=',scaledX)
    # print(scaler.fit(data))

    # print('mean X=',scalerX.mean_)
    return scaler, scaled


def PredictDistribution(hist3):
    global data, fname, label

    # # scalerY = readVar(fy)
    # # scalerX = readVar(fx)
    # PATH = 'ModelKFold_3.pt'
    # model = torch.nn.Sequential(
    #     torch.nn.Linear(7, 256),
    #     torch.nn.BatchNorm1d(256, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(256, 100),
    #     torch.nn.BatchNorm1d(100, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(100, 100),
    #     torch.nn.BatchNorm1d(100, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(100, 100),
    #     torch.nn.BatchNorm1d(100, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(100, 32),
    #     torch.nn.Dropout(p=0.2),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(32, 7),
    #     torch.nn.Tanh()
    # )

    # PATH = 'ModelKFold_v2_3.pt'
    #
    # model = torch.nn.Sequential(
    #     torch.nn.Linear(7, 256),
    #     torch.nn.BatchNorm1d(256, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(256, 100),
    #     torch.nn.BatchNorm1d(100, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(100, 100),
    #     torch.nn.BatchNorm1d(100, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(100, 100),
    #     torch.nn.BatchNorm1d(100, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(100, 32),
    #     torch.nn.BatchNorm1d(32, affine=False),
    #     torch.nn.LeakyReLU(),
    #     torch.nn.Linear(32, 7)
    #
    # )
    # PATH = 'ModelKFold_v2_Normalizer1.pt'
    PATH='Model.pt'

    model = torch.nn.Sequential(
        torch.nn.Linear(7, 256),
        torch.nn.BatchNorm1d(256, affine=False),
        torch.nn.LeakyReLU(),
        #             torch.nn.Sigmoid(),
        torch.nn.Linear(256, 100),
        torch.nn.BatchNorm1d(100, affine=False),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 32),
        torch.nn.BatchNorm1d(32, affine=False),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(32, 7)
        #             torch.nn.Sigmoid()
        #             torch.nn.Tanh()
    )

    device = 'cpu'
    model.load_state_dict(torch.load(PATH, map_location=device))
    model.eval()
    # x=scaledX
    #
    #
    # y = scaledY
    # y=torch.from_numpy(y.astype(np.float32))
    # xx, normalisedX=data_x_normalize(hist3)
    # input= torch.from_numpy(normalisedX.astype(np.float32).reshape(1, -1))
    # # input=input1.reshape(1, -1)
    # prediction = model(input)
    #
    # data=scalerY.inverse_transform(prediction.detach().numpy())
    # # data=out1[myDataIndex]
    # label='Histogram of predicted sinter particle'
    # plot_Histogram_intermsof_weight(data,label)
    # plt.savefig('Predicted.png')
    # hist3[3] = hist3[3] * 1.25

    # normalisedX = scalerX.transform(hist3.reshape(1,-1))
    # # print(xxx)
    #
    #
    # # xx, normalisedX=data_x_normalize(xxx)
    # input= torch.from_numpy(normalisedX.astype(np.float32))
    # # input=input1.reshape(1, -1)
    # prediction = model(input)
    #
    # data=scalerY.inverse_transform(prediction.detach().numpy())
    # # print(data)
    # # data=out1[myDataIndex]
    # fname=mySet+'Predicted.png'
    # label='Histogram of predicted sinter particle'
    # plt=plot_Histogram_intermsof_weight(data[0],label,fname)
    # # plt.savefig('Predicted.png')

    # fx = 'normx.pkl'
    # fy = 'normy.pkl'
    fx = 'normxNormalizer.pkl'
    fy = 'normyNormalizer.pkl'
    scalerYE = readVar(fy)
    scalerXE = readVar(fx)

    sampledata = hist3
    myweight = [2.5, 7.5, 12.5, 20, 32.5, 45, 60]
    w1 = torch.from_numpy(np.array(myweight))
    # print(w1)
    total_w = torch.sum(w1, dim=0)  # size = [1, ncol]
    # print(type(myweight))
    # print(type(w1))
    # print(total_w)
    x_data = torch.from_numpy(sampledata) * w1
    # print(x_data[0:5])

    New_data_x = x_data / total_w
    # print(New_data_x)
    # print('hist3=', hist3)
    # print(hist3)
    # hist3[3]=hist3[3]+(hist3[3]*0.25)
    # print('hist3=', hist3)
    normalisedX = scalerXE.transform(New_data_x.reshape(1, -1))
    # print(xxx)
    # xx, normalisedX=data_x_normalize(xxx)
    input1 = torch.from_numpy(normalisedX.astype(np.float32))
    # input=input1.reshape(1, -1)
    prediction = model(input1)
    data = scalerYE.inverse_transform(prediction.detach().numpy())
    # print(data)
    # data=out1[myDataIndex]
    # fname = mySet + 'Predicted.png'
    fname = 'Predicted.png'
    label1 = 'Histogram of predicted sinter particle'
    plt, bins, data = plot_Histogram_intermsof_weight(data[0], label1, fname)
    # plt.show()

    return plt, bins, data


def AnalyseRegionProperty(image):
    global label, data, Data, fname
    # region prop analysis
    from skimage.measure import label, regionprops, regionprops_table
    label_img = label(image)
    regions = regionprops(label_img)
    coll_area = []
    coll_major_axis = []
    coll_minor_axis = []
    for props in regions:
        area1 = props.area
        coll_area.append(area1)
        coll_major_axis.append(props.major_axis_length)
        coll_minor_axis.append(props.minor_axis_length)
    ###############
    # defining conversion and adjustment from pixel to mm
    # adjustment=2.5
    # adjustment=2
    adjustment = 1.25
    #
    # adjustment=2.5 ## last working on 19-05-2021
    # adjustment=1
    # adjustment=7.5
    # conv=1045/3500
    # conv=1894/9424
    conv = 1045 / 5200
    # print(conv)
    conv = conv * adjustment
    # adjustment=1
    # print(conv)
    ############
    # computing equvalent radius
    data = np.array(coll_area) * conv * conv
    # print(data)
    equRadius = np.sqrt(data / 3.14)
    #######################
    # plt.ylabel('Frequency');
    # plt.xlabel('Major_axis in terms of mm');
    ###############
    # creating loc
    # print('mySet=', mySet)
    # flag = 'log_fname.csv'
    # mtlist = [mySet]
    # Append_log((mtlist), flag)

    # bins = [0, 5, 10, 15, 25, 40, 50, 500]  # your bins
    # xLabel = 'Count'
    # yLabel = 'Major_axis in terms of mm'
    # Data = coll_major_axis
    # fname = mySet + 'major.png'
    # hist1 = plot_Unequal_Bin_Histogram(bins, xLabel, yLabel, Data, conv, fname)
    # print('hist1=', hist1)
    # flog = 'log_major.csv'
    # Append_log(hist1, flog)
    # bins = [0, 5, 10, 15, 25, 40, 50, 500]  # your bins
    # xLabel = 'Count'
    # yLabel = 'Minor_axis in terms of mm'
    # Data = coll_minor_axis
    # fname = mySet + 'minor.png'
    # hist2 = plot_Unequal_Bin_Histogram(bins, xLabel, yLabel, Data, conv, fname)
    # print('hist2=', hist2)
    # flog = 'log_minor.csv'
    # Append_log(list(hist2), flog)
    bins = [0, 5, 10, 15, 25, 40, 50, 500]  # your bins
    xLabel = 'Count'
    yLabel = 'Equivalent Dia in terms of mm'
    Data = equRadius * 2
    conv = 1
    # fname = mySet + 'Equivalent.png'
    fname = 'Equivalent.png'
    hist3 = plot_Unequal_Bin_Histogram(bins, xLabel, yLabel, Data, conv, fname)
    # print('hist3=', hist3)
    # flog = 'log_Equivalent.csv'
    # Append_log(hist3, flog)
    return hist3


def Extract_ROI(mylist):
    # global image
    dst1 = mylist[0][0]
    dst2 = mylist[1][0]
    dst3 = mylist[2][0]
    dst4 = mylist[3][0]
    ## Merging Different Images
    I = combine_image(dst1, dst2, dst3, dst4)
    # filename = mySet + 'crop.jpg'
    # cv2.imwrite(filename, I)
    ## Preparind data for boundary detection
    se = disk(2)
    Ie = erosion(I, se)
    Iobr = imreconstruct(Ie, I)
    Iobrd = dilation(Iobr, se)
    Iobrcbr = imreconstruct(imcomplement(Iobrd), imcomplement(Iobr))
    Iobrcbr = imcomplement(Iobrcbr)
    image = Iobrcbr
    block_size = 89
    thresh = threshold_mean(image)
    binary_local = image > thresh
    ab = binary_closing(binary_local)
    ab = binary_closing(ab)
    seD = disk(3)
    ab = binary_opening(ab, seD)
    ab = binary_opening(ab, seD)
    seD = disk(2)
    ab = binary_closing(ab, seD)
    # filename = mySet +'bw.jpg'
    # plt.imsave(filename, np.array(ab), cmap=cm.gray)
    # print("saved")
    ############################################## mks
    # img = cv2.imread(mySet+'crop.jpg')
    #
    # img1 = cv2.imread(mySet+'bw.jpg')
    # img2=img1[:,:,0]
    # thresh=img2
    ##################
    # img = I
    img = cv2.merge((I, I, I))
    # img1 = cv2.imread(mySet+'bw.jpg')
    img1 = np.array(ab)
    # img2=img1[:,:,0]
    thresh = np.uint8(ab)
    #
    ###################
    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    # opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 1)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel,
                               iterations=1)  # mks try changing value to 7 for big image and see
    # opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 7) # mks try changing value to 7 for big image and see
    ############################################ main function
    # sure background area
    # sure_bg = cv2.dilate(opening,kernel,iterations=3)
    sure_bg = cv2.dilate(opening, kernel, iterations=7)
    # Finding sure foreground area
    # dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)
    # dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)  #mks try changing value to 7 for big image and see
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)  # mks try changing value to 7 for big image and see
    # print('dist_transform.max()=',dist_transform.max())
    # ret, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)
    ret, sure_fg = cv2.threshold(dist_transform, 0.1 * dist_transform.max(), 255, 0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    ##########################################
    # find markers
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0
    ##############################
    # ## marking on original image
    markers2 = cv2.watershed(img, markers)
    img[markers2 == -1] = [255, 0, 0]
    ###########
    # extracting and visualizing boundary information from marker
    img3 = np.zeros((img.shape), np.uint8)
    img3[markers2 <= -1] = [255, 255, 255]
    #############
    # extracting ROI with marker of ROI
    img4 = np.zeros((img.shape), np.uint8)
    markers4 = cv2.watershed(img4, markers)
    img4[markers4 <= 1] = [255, 255, 255]
    img = 255 - img4
    ############################## mks attenstion required
    # filename = mySet+'marked2.jpg'
    # cv2.imwrite(filename, img)
    ##########################
    # processing binary mark for image analysis
    # img = cv2.imread(mySet+'marked2.jpg')
    ####################
    ##########################
    # processing binary mark for image analysis
    img1 = np.uint8(img)
    image1 = img1[:, :, 0]
    im2 = area_opening(image1)
    se = disk(1)
    image = binary_erosion(im2, se)
    se = disk(1)
    image = binary_opening(image, se)
    min_size = 20
    # min_size=5
    image = remove_small_objects(image, min_size)
    return image


def CorrectImage(img1, img2, img3, img4):
    # global mylist
    que = Queue()  # Python 3.x
    threads_list = list()
    # fname1 = 'Cycle2_bottom_1.jpg'
    # fname1 = [s for s in arr if "bottom" in s][0]
    # t = Thread(target=lambda q, arg1: q.put(PerspectiveCorrectionMultiThread(arg1)), args=(que, fname1))
    t = Thread(target=lambda q, arg1: q.put(PerspectiveCorrectionMultiThread(arg1)), args=(que, img1))
    t.start()
    threads_list.append(t)
    # fname2 = 'Cycle2_left_1.jpg'
    # fname2 = [s for s in arr if "left" in s][0]
    t2 = Thread(target=lambda q, arg1: q.put(PerspectiveCorrectionMultiThread(arg1)), args=(que, img2))
    t2.start()
    threads_list.append(t2)
    # fname3 = 'Cycle2_right_1.jpg'
    # fname3 = [s for s in arr if "right" in s][0]
    t3 = Thread(target=lambda q, arg1: q.put(PerspectiveCorrectionMultiThread(arg1)), args=(que, img3))
    t3.start()
    threads_list.append(t3)
    # fname4 = 'Cycle2_top_1.jpg'
    # fname4 = [s for s in arr if "top" in s][0]
    t4 = Thread(target=lambda q, arg1: q.put(PerspectiveCorrectionMultiThread(arg1)), args=(que, img4))
    t4.start()
    threads_list.append(t4)
    # Join all the threads
    for t in threads_list:
        t.join()
    # Check thread's return value
    # i=0
    mylist = []
    while not que.empty():
        result = que.get()
        mylist.append(result)

    return mylist




def ConfigVariable():
    global file, conf_list, status_for_crop, status_for_WindowSize, DescCrop_y, DescCrop_x, DescCrop_h, DescCrop_w, inputSubFolder, outputSubFolder, resultDir
    with open(r'config.yaml') as file:
        conf_list = yaml.load(file, Loader=yaml.FullLoader)
    inputFolderName = conf_list['var']['inputFolderName']
    outputFolderName = conf_list['var']['outputFolderName']
    DataDirectory = conf_list['var']['DataDirectory']
    myroot = conf_list['var']['myroot']
    status_for_crop = conf_list['DescCrop']['status']
    status_for_WindowSize = conf_list['DescWindowSize']['windowSize']
    DescCrop_y = conf_list['DescCrop']['y']
    DescCrop_x = conf_list['DescCrop']['x']
    DescCrop_h = conf_list['DescCrop']['h']
    DescCrop_w = conf_list['DescCrop']['w']
    inputSubFolder = DataDirectory + "/" + inputFolderName
    outputSubFolder = DataDirectory + "/" + outputFolderName
    resultDir = []
    for x in os.walk(inputSubFolder):
        resultDir = x[1]
        break


# Main image analysis function
def ImageAnalysis(img1, img2, img3, img4):
    ''' This function takes 4 images and process them to analyse the sinter particle

    :param img1: first image path
    :param img2: second image path
    :param img3: third image path
    :param img4: fourth image path
    :return:
        bins: bin used for histogram
        data: original sample to plot histogram
    '''
    # global plt, data
    print("Step : 1 Geometric correction of images......... ", end='')
    mylist = CorrectImage(img1, img2, img3, img4)
    print("   Done")
    #################
    print("Step : 2 Image analysis ......... ", end='')
    image = Extract_ROI(mylist)
    print("   Done")
    print("Step : 3 Histogram computation......... ", end='')
    hist3 = AnalyseRegionProperty(image)
    plt, bins, data = PredictDistribution(hist3)
    print("   Done")

    return bins, data

# Read setting from config file
ConfigVariable()

AttributeError: ignored